# Comparing Hamiltonian with Nayra

In [ ]:
%matplotlib notebook

import pathlib
from scipy import interpolate as interp
import pythtb as pytb
import scipy.linalg as la

import sim_tb as stb
import bz_utilities as bzu
import toy_models as toy
import plot_utils as pltu
import linear_response as lr

In [ ]:
def reorder_matrix(H):
    """
    Take a (norb, 2, norb, 2) matrix and returns
    a (2*norb, 2*norb) matrix. Following convention:
    [orb1_up, orb1_down, orb2_up, orb2_down, ....]
    """
    norb = np.shape(H)[0]
    new_H = np.zeros((2*norb, 2*norb), dtype="complex")
    t_arr = [np.array([0,0]),
            np.array([0.5, 0]),
            np.array([0, 0.5])]
    for i1 in range(norb):
        for i2 in range(norb):
            for s1 in range(2):
                for s2 in range(2):
                    n1 = 2 * i1 + s1
                    n2 = 2 * i2 + s2
                    delta_t = t_arr[i1] - t_arr[i2]
                    #exp = np.exp(2j * np.pi*kpt@delta_t)
                    h = H[i1, s1, i2, s2]
                    new_H[n1, n2] = h
    return new_H

def reorder_v_operator(v, H, kpt):
    """
    """
    norb = np.shape(H)[0]
    new_v = np.zeros((2, 2*norb, 2*norb), dtype="complex")
    t_arr = [np.array([0,0]),
            np.array([0.5, 0]),
            np.array([0, 0.5])]
    a1 = np.array([1,0])
    a2 = np.array([0.5, 0.5*np.sqrt(3)])
    for i1 in range(norb):
        for i2 in range(norb):
            for s1 in range(2):
                for s2 in range(2):
                    n1 = 2 * i1 + s1
                    n2 = 2 * i2 + s2
                    delta_t = t_arr[i1] - t_arr[i2]
                    delta_t_re = delta_t[0]*a1 + delta_t[1]*a2
                    exp = np.exp(2j * np.pi*kpt@delta_t)
                    h = H[i1, s1, i2, s2]
                    vx = v[0, i1,s1, i2, s2]
                    vy = v[1, i1,s1, i2, s2]
                    new_v[0, n1, n2] = vx * exp + 1j*h * exp * delta_t_re[0]
                    new_v[1, n1, n2] = vy * exp + 1j*h * exp * delta_t_re[1]
    return new_v
                    
    

# Defining simulations without/with SOC and K-points

t, J, t2 = 1.0, 1.7, 0.2
mag_mode = "coplanar"
path = toy.init_kagome_model(t, J, t2, mag_mode)
Sim = stb.Simulation_TB(path)
t, J, t2 = 1.0, 1.7, 0.0
path = toy.init_kagome_model(t, J, t2, mag_mode)
Sim0 = stb.Simulation_TB(path)
    
M = bzu.cartesian_to_reduced_k_matrix(Sim0.rlat[0], Sim0.rlat[1])
K = M @ np.array([2.094, 3.63])
Kp = M @ np.array([-2.94, 3.63])
PR = M @ np.array([0.23, 1.87])
G = np.array([0,0])

points = {"K":K, "Kp":Kp, "PR":PR,"G":G}

folder = toy.ROOT_DIR  + "H_components/"
toy.mk_dir(folder)

In [ ]:
for mode in ["no_soc", "soc"]:
    for point in ["K", "Kp", "PR", "G"]:
        cSim = {"no_soc":Sim0, "soc": Sim}[mode]
        kpt = points[point]
        name = "H_{}_{}.npy".format(mode,point)
        name_v = "velocity_{}_{}.npy".format(mode,point)
        H0 = cSim.model._gen_ham(k_input=kpt)
        v0 = cSim.velocity_operator(kpt)
        H = reorder_matrix(H0, kpt)
        v = reorder_v_operator(v0, H0, kpt)
        np.save(folder+name, H)
        np.save(folder+name_v, v)

In [ ]:
mode = "soc"
cSim = {"no_soc":Sim0, "soc": Sim}[mode]
kpt = points["PR"]
Hk_ = cSim.model._gen_ham(k_input=kpt)
Hk = reorder_matrix(Hk_)
Berry_k = cSim.berry_curvature(kpt[0], kpt[1], 0, 0, 1, mode="re")
Berry_k